In [2]:
import pandas as pd
import seaborn as sns
from pathlib import Path

# dry = "size_counters/220701-152907"
bench = Path("size_counters/220819-151202")

# in minutes
times = [60]
points = list(range(3))


def diff(a: pd.DataFrame, b: pd.DataFrame) -> pd.DataFrame:
    out = b.copy()
    out[out.columns[2:]] -= a[a.columns[2:]]
    return out


def total(d: pd.DataFrame) -> pd.DataFrame:
    return d.loc[:, d.columns != "CPU"].groupby(["op"]).sum()


benches = [[pd.read_csv(bench / f"t{t}" / f"t{p}.csv")
            for p in points] for t in times]


In [3]:
print(total(benches[0][0]))
print(total(diff(benches[0][0], benches[0][1])))
print(total(diff(benches[0][1], benches[0][2])))

    bulk      0     1    2    3  4  5  6  7  8   9    10
op                                                      
a    597  72611  1236  701  292  8  2  3  4  1  10     0
f      0  44128  1058   12   26  2  2  3  3  4   6  8022
    bulk     0   1  2  3  4  5  6  7  8  9  10
op                                            
a     15  2538  34  3  9  0  0  0  0  0  1   0
f      0  1656  29  0  4  0  0  0  0  0  0   0
    bulk      0   1       2      3  4  5  6  7  8  9  10
op                                                      
a      9  84672  55  103712  71617  0  0  0  0  0  1   0
f      0  87305  73  103692  71569  0  0  0  0  0  1   0


In [ ]:
print(total(benches[1][0]))
print(total(diff(benches[1][0], benches[1][1])))
print(total(diff(benches[1][1], benches[1][2])))

In [ ]:
print(total(benches[2][0]))
print(total(diff(benches[2][0], benches[2][1])))
print(total(diff(benches[2][1], benches[2][2])))

In [ ]:
print(total(benches[3][0]))
print(total(diff(benches[3][0], benches[3][1])))
print(total(diff(benches[3][1], benches[3][2])))